In [1]:
import sys
sys.path.insert(0, ".")


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from core import fitting
from core import scraping, mapping

In [4]:
from prediksicovidjatim import database, config, util
from prediksicovidjatim.data.model import ModelDataRepo

In [5]:
K_TEST = os.getenv("K_TEST")

def init():
    database.init()
    config.init_plot()
    with database.get_conn() as conn, conn.cursor() as cur:
        ModelDataRepo.init_weights(cur)
        mapping.init(cur)
    

In [6]:
init()

from prediksicovidjatim.data.map import MapDataRepo
kabko = MapDataRepo.fetch_kabko_need_mapping(None)
kabko[0]

kabko = [("KAB. BANGKALAN", 70)]

from prediksicovidjatim.mapping import updater
mapping.cache_geometry()
len(updater.GEOMETRY_CACHE)

u = mapping.get_updater()
layer = u.get_layer(mapping.REAL_LAYER_ID)
u.cache_kabko_geometry(layer, util.format_date(mapping.FIRST_TANGGAL_STR))
from prediksicovidjatim.mapping import updater
len(updater.GEOMETRY_CACHE)

In [8]:
mapping.update_map_all()
util.lprofile.print_stats()

38 kabko needs updating
Caching kabko geometry


KeyboardInterrupt: 

fitting.fit_all(kabko=["KAB. BANGKALAN", "KOTA SURABAYA", "KOTA BLITAR", "KAB. SAMPANG"], max_process_count=2)
util.lprofile.print_stats()

In [ ]:
import gc
gc.collect()